In [2]:
!pip install dataset
import requests
import dataset
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

In [17]:
db =  dataset.connect('sqlite:///quotes.db')

authors_seen = set()  #空集合

base_url = 'http://quotes.toscrape.com/'

def clean_url(url):
        #'/author/Steve-Martin'を'Steve-Martinに変更してCleanにする
        #まず、urljoinを使って絶対URLにする
        url = urljoin(base_url, str(url))
        #urlparseを使ってpathの部分を取り出す
        path = urlparse(url).path
        #'/author/以下がpathとして出てくる'
        
        #'/'でpathを分割、２番目の値を取得
        return path.split('/')[1]

def scrape_quotes(html_soup):
        quotes = html_soup.select('div.quote')
        for quote in quotes:
                quote_text = quote.find(class_ = 'text').get_text(strip = True)
                quote_author_url = clean_url(quote.find(class_ = 'author').find_next_sibling('a').get('href'))
                quote_tag_urls = [clean_url(a.get('href') for a in quote.find_all('a', class_ = 'tag'))]
                authors_seen.add(quote_author_url)
                
                #この名言とタグを保存する
                quote_id = db['quotes'].insert(
                    {'text': quote_text, 'author': quote_author_url})
                db['quote_tags'].insert_many(
                [{'quote_id': quote_id, 'tag_id': tag} for tag in quote_tag_urls])

def scrape_authors(html_soup, author_id):
        author_name = html_soup.find(class_ = 'author-title').get_text(strip = True)
        author_born_date = html_soup.find(class_ = 'author-born-date').get_text(strip = True)
        author_born_loc = html_soup.find(class_ = 'author-born-location').get_text(strip = True)
        author_discription = html_soup.find(class_ = 'author-description').get_text(strip = True)
        db['authors'].insert({
            'author_id': author_id, 'name': author_name, 'born_date': author_born_date, 'born_location': author_born_loc, 'description': author_discription
        })

#まず全てのページの名言をスクレイピングする
url = base_url
while True:
        print('Now...', url)
        res = requests.get(url)
        html_soup = BeautifulSoup(res.text, 'html.parser')
        
        #名言をスクレイピング
        scrape_quotes(html_soup)
        
        #次のページがあるか
        next_a = html_soup.select('li.next > a')
        if not next_a or not next_a[0].get('href'):
                break
        url = urljoin(url, next_a[0].get('href'))
        
#発言者情報を取り出す
for author_id in authors_seen:
        url = urljoin(base_url, '/author/' + author_id)
        print('Now...', url)
        res = requests.get(url)
        html_soup = BeautifulSoup(res.text, 'html.parser')
        
        #発言者情報を拾う
        scrape_authors(html_soup, author_id)

Now... http://quotes.toscrape.com/
Now... http://quotes.toscrape.com/page/2/
Now... http://quotes.toscrape.com/page/3/
Now... http://quotes.toscrape.com/page/4/
Now... http://quotes.toscrape.com/page/5/
Now... http://quotes.toscrape.com/page/6/
Now... http://quotes.toscrape.com/page/7/
Now... http://quotes.toscrape.com/page/8/
Now... http://quotes.toscrape.com/page/9/
Now... http://quotes.toscrape.com/page/10/
Now... http://quotes.toscrape.com/author/author
